In [106]:
from StrategyHandler import *
import pandas as pd
import sqlite3
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
from Production import Production
from functools import reduce

In [107]:
strategy=Strategy('btcusdt')
con = sqlite3.connect('market.db', timeout=10)

In [108]:
# p=Production()
# p.strategy.get_data(8000, 'btcusdt_1day_candle', 'btcusdt_60min_candle', 'btcusdt_30min_candle','btcusdt_15min_candle', 'btcusdt_5min_candle', 'btcusdt_1min_candle')
# freq=['1min','5min','15min','30min','60min','1day']
# period=[5,10,20,30,60]
# for x in freq:
#     for y in period:
#         p.boll_reverse_system(x,y)

In [109]:
pnl_list = []
for item in p.strategy_list:
    pnl = p.strategy_list[item][0][['pnl']]
    pnl.columns = [item]
    pnl_list.append(pnl)
    pnl = reduce(lambda x, y: pd.merge(x, y, on='id', how='outer'), pnl_list)
    pnl = pnl.sort_index()
    pnl = pnl.fillna(0)
    cov = pnl.tail(24000).cumsum().cov()
    x = 1 / (cov.sum(axis=1))
    x = x / sum(x)

In [110]:
p=Production()
p.strategy.get_data(20000, 'btcusdt_1day_candle', 'btcusdt_60min_candle', 'btcusdt_30min_candle','btcusdt_15min_candle', 'btcusdt_5min_candle', 'btcusdt_1min_candle')
p.ma_system("1day", 10, 20)
p.ma_system("60min", 10, 20)
p.boll_system("60min",30)
p.boll_system("1day",30)
p.boll_reverse_system("5min",30)
p.boll_reverse_system("15min",30)
portfolio=p.get_portfolio()

updated strategy:ma_system_1day_1020 at23:22:58
updated strategy:ma_system_60min_1020 at23:22:59
updated strategy:boll_system_60min_30 at23:23:01
updated strategy:boll_system_1day_30 at23:23:02
updated strategy:boll_reverse_system_5min_30 at23:23:07
updated strategy:boll_reverse_system_15min_30 at23:23:10


In [111]:
# p.strategy_list['ma_system_1day_1020'][0]['pnl'].cumsum().plot()

In [112]:
p.evaluate_strategy()

ma_system_1day_1020 Est Fee:0.138
ma_system_1day_1020 Total Return:3.34
ma_system_1day_1020 Sharp Ratio:4.46
ma_system_60min_1020 Est Fee:0.298
ma_system_60min_1020 Total Return:0.44
ma_system_60min_1020 Sharp Ratio:1.71
boll_system_60min_30 Est Fee:0.14
boll_system_60min_30 Total Return:1.41
boll_system_60min_30 Sharp Ratio:3.54
boll_system_1day_30 Est Fee:0.09
boll_system_1day_30 Total Return:4.33
boll_system_1day_30 Sharp Ratio:4.51
boll_reverse_system_5min_30 Est Fee:0.316
boll_reverse_system_5min_30 Total Return:0.55
boll_reverse_system_5min_30 Sharp Ratio:3.22
boll_reverse_system_15min_30 Est Fee:0.164
boll_reverse_system_15min_30 Total Return:0.52
boll_reverse_system_15min_30 Sharp Ratio:1.8


In [113]:
def get_performance(portfolio):
    trade=pd.read_sql('select * from btcusdt_trade_log', con)
    trade['direction']=((trade['OrderType'].str[0]=='b')*2-1)
    trade['position']=(trade['direction']*trade['Amount'].astype(float)).cumsum()
    trade['liability']=-(trade['Amount'].astype(float)*trade['Price'].astype(float)*trade['direction']).cumsum()
    portfolio=portfolio.reset_index()
    portfolio['id']*=1000
    portfolio.rename(columns={ portfolio.columns[0]: "Time" }, inplace = True)
    result=portfolio.merge(trade[['Time','position','Fee','liability']], how='outer', on='Time')
    performance=result.sort_values('Time')
    performance[['price','position_x','position_y','liability']]=performance[['price','position_x','position_y','liability']].fillna(method='ffill')
    performance=performance.fillna(0)
    performance['asset']=performance['position_y']*performance['price']
    performance['equity']=performance['asset']+performance['liability']
    performance['Fee']=performance['Fee'].astype(float).cumsum()*2
    performance['Time']=pd.to_datetime(performance['Time']/1000,unit='s')+pd.Timedelta('08:00:00')
    return performance

In [ ]:
def plot_result(data):
    fig, ax1 = plt.subplots()
    fig.set_size_inches(18.5, 10.5, forward=True)

    ax1.set_xlabel('Time')
    ax1.set_ylabel('price')
    ax1.plot(data['Time'], data['price'],color='peru',linewidth=0.8)
    ax1.scatter(data['Time'],data['price'],c=data['position_x'],s=2)
    
    ax2 = ax1.twinx()
    ax2.set_ylabel('position')
    ax2.set_ylabel('pnl')
    ax2.plot(data['Time'],data['pnl'].cumsum(),color='maroon') 
    
    ax3= ax1.twinx()
    ax3.spines["right"].set_position(("axes", 1.1))
    ax3.set_ylabel('equity')
    ax3.plot(data['Time'],data['equity'],color='grey')
#     ax3.plot(data['Time'],data['Fee'])

    number_trades=sum(data['position_x'].diff()!=0)
    print("Est Fee:" + str(number_trades*0.002))
    print("Total Return:"+ str(round(data['pnl'].sum(),2)))
    print("Sharp Ratio:"+ str(round(data['pnl'].sum()/data['pnl'].cumsum().std(),2)))

x=get_performance(portfolio)
plot_result(x.tail(20000))

In [ ]:
x.tail(50)